In [1]:
import openai
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Configurar OpenAI
api_key = os.getenv("TOKEN")
client = openai.OpenAI(api_key=api_key)

# Configurar conexión a MySQL
def obtener_conexion_mysql():
    return mysql.connector.connect(
        host=os.getenv('HOST'),
        user=os.getenv('USERNAME'),
        password=os.getenv('PASSWORD'),
        database=os.getenv('DATABASE')
    )

# Descripción del esquema de la base de datos
esquema_bd = """
La base de datos tiene las siguientes tablas:

1. **pnrp.airflow_hexing_ulti_comu**: Contiene información sobre comunicaciones de los medidores Hexing.
    - **CLAVE**: Clave del cliente propietario del medidor.
    - **MEDIDOR**: Identificador del medidor.
    - **FECHA**: Fecha de la comunicación.
    - **LECTURA**: Lectura del medidor.

2. **pnrp.airflow_hexing_alarmas**: Contiene información sobre alarmas de los medidores Hexing.
   - **CLAVE**: Clave del cliente propietario del medidor.
   - **MEDIDOR**: Identificador del medidor.
   - **ALARM_CODE**: Código de la alarma.
   - **ALARM_DESC**: Nombre de la alarma.
   - **FECHA**: Fecha de la alarma.

5. **pnrp.bitacora_ac**: Contiene registros de tickets de orden de servicio, análisis y otros detalles.
   - **TICKET**: Número de ticket.
   - **CLAVE**: Clave del cliente propietario del medidor.
   - **MEDIDOR**: Identificador del medidor.
   - **NO_CONFORMIDAD**: No conformidad identificada.
   - **REQUIERE_OS**: Indica si requiere orden de servicio (0 o 1).
   - **COMENTARIO_ANALISTA**: Comentarios del analista.
   - **ES_RECURRENTE**: Indica si el problema es recurrente (0 o 1).
   - **ANOMALIA**: Anomalía reportada.
   - **CIRCUITO**: Circuito del medidor.
   - **ESTADO**: Estado del ticket.

NOTA: Simpre que se pida tickets se debe regresar el número del ticket incluso si no lo especifica la pregunta. Las 3 tablas se pueden relacionar por la clave y el medidor.
"""

# Función para generar SQL desde lenguaje natural
def query_natural_language_to_sql(user_query):
    prompt = f"""
    Eres un asistente especializado en generación de consultas SQL a partir de lenguaje natural.
    El esquema de la base de datos es el siguiente:
    {esquema_bd}
    
    Convierte la siguiente consulta en lenguaje natural a SQL para una base de datos MySQL. 
    Genera únicamente el código SQL, sin comentarios ni marcas de código adicionales. Consulta: {user_query}
    """
    
    completion = client.chat.completions.create(
        model="ft:gpt-4o-mini-2024-07-18:personal:modelo-hexing-fine-tuning:A8rAJQ0R",  # Cambia al modelo específico si es necesario
        messages=[
            {"role": "system", "content": "Eres un asistente especializado en ingeniería eléctrica y detección de hurto de energía."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0
    )
    
    sql_query = completion.choices[0].message.content.strip().replace('```', '')
    return sql_query

# Función para ejecutar la consulta SQL en MySQL
def ejecutar_consulta_mysql(query_sql):
    try:
        conexion = obtener_conexion_mysql()
        cursor = conexion.cursor()
        cursor.execute(query_sql)
        resultados = cursor.fetchall()
        return resultados
    except Error as e:
        print(f"Error al ejecutar la consulta: {e}")
        return []
    finally:
        if conexion.is_connected():
            cursor.close()
            conexion.close()

# Función para interpretar resultados usando el modelo de OpenAI
def interpretar_resultados_con_modelo(resultados, consulta_usuario):
    prompt = f"""
    Eres un asistente especializado en ingeniería eléctrica y detección de hurto de energía, especializado en interpretación de datos.
    Los resultados de la consulta SQL para la pregunta "{consulta_usuario}" son los siguientes:
    {resultados}
    
    Proporciona una interpretación en lenguaje natural de estos resultados, incluyendo detalles relevantes y contexto cuando sea necesario, basado en tu información previamnete cargada. Si la consulta sql es muy larga explica solo las 3 partes más importantes o críticas.
    """
    
    completion = client.chat.completions.create(
        model="ft:gpt-4o-mini-2024-07-18:personal::A3UiuJ83",  # Cambia al modelo específico si es necesario
        messages=[
            {"role": "system", "content": "Eres un asistente especializado en ingeniería eléctrica y detección de hurto de energía, especializado en interpretación de datos. "},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.5
    )
    
    interpretacion = completion.choices[0].message.content.strip()
    return interpretacion

# Función principal para manejar la consulta en lenguaje natural y la interpretación
def manejar_consulta_natural(usuario_query):
    consulta_sql = query_natural_language_to_sql(usuario_query)
    # print(f"Consulta SQL generada: {consulta_sql}")  # Mostrar la consulta SQL generada
    resultados = ejecutar_consulta_mysql(consulta_sql)
    
    # Interpretar los resultados usando el modelo
    respuesta_interpretada = interpretar_resultados_con_modelo(resultados, usuario_query)
    
    return respuesta_interpretada

# Ejemplo de uso
# consulta_usuario = "Al medidor '2017-012-044283' se le han generado tickets? cual es el estado del ticket, y requirio orden de servicio?"
consulta_usuario = "recuento de la cantidad de alarmas que presento el medidor 000016347516 en enero 2024 y menciona las más importantes"
# consulta_usuario = "la clave '1236290' es un cliente recurrente?"
# consulta_usuario = "hay medidores recurrentes en el circuito 'BER288' "
# consulta_usuario = "que alarmas presento el medidor 000020844126 entre las fechas 11/08/2024 y 24/08/2024"
# consulta_usuario = "tiene la clave 5074550 tuvo el ticket TK-HEX-9972 cual fue el estado del ticket y que alarmas presento entre el periodo inicial y final del ticket "

respuesta = manejar_consulta_natural(consulta_usuario)

# Mostrar la respuesta interpretada
print("Respuesta en lenguaje natural:")
print(respuesta)


Respuesta en lenguaje natural:
Los resultados de la consulta SQL indican un total de 30 alarmas de subvoltaje en el medidor 000016347516 durante enero de 2024, lo que sugiere un problema de suministro eléctrico o una mala configuración del medidor. Aquí están las interpretaciones más relevantes:

1. **Subvoltaje**: Las alarmas de subvoltaje son las más frecuentes, con 30 reportes cada una para las fases L1, L2 y L3. Esto puede ser indicativo de un problema en la red que está causando que el voltaje caiga por debajo de lo normal. La subvoltaje puede ser provocada por un aumento en la demanda, reducción en la oferta o problemas en la infraestructura de la red. Es crucial que se investigue la causa de esta subvoltaje, ya que puede afectar la calidad del servicio y la vida útil del equipo.

2. **Acciones de potencia**: Se reportaron 20 alarmas de "Power Down (Long Power Failure)", lo que indica que el medidor experimentó un apagón prolongado. Esto podría ser un signo de un problema en la r

In [79]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("TOKEN")
client = openai.OpenAI(api_key=api_key)

messages = [
    {
        "role": "system",
         "content": "Eres un asistente especializado en ingeniería eléctrica y detección de hurto de energía. "
            "Tu tarea es, a partir de las alarmas con sus fechas y los comentarios históricos de analistas, "
            "que pueden incluir datos como la clave del dispositivo, la fecha de análisis, "
            "el comentario del analista, el tipo de medida y el circuito, "
            # "Si solo el medidor presenta caída de consumo y no todo el circuito, probablemente sea un hurto de energía, y requiera orden de servicio "
            "Menciona la criticidad"
            "Ten en cuenta la criticidad de las alarmas de los medidores para determinar si se requiere una orden de servicio, "
            # "Si el medidor no comunica, probablemente sea un problema de comunicación y requiera orden de servicio"
            "explica las alarmas que se presentan en el medidor y si se requiere una orden de servicio, "
            "y proporcionar una justificación adecuada en tus respuestas en por lo menos 40 palabras."
    },
    {
        "role": "user",
        "content": "La clave 1911628 del circuito RET289, presenta:"
                    "Presenta las siguientes alarmas:"
                    "Manually Demand Reset"
                    "Current Unbalance End"
                    "Missing Voltage L3 Start"
                    "¿Requiere una orden de servicio?"
                    "¿Este cliente es recurrente historicamente?"
                    "¿Cuál es la criticidad?"
    }
]


completion = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal::A3UiuJ83",  # Especifica tu modelo fine-tuned, MOdelo que mejor resultados ha dado ft:gpt-4o-mini-2024-07-18:personal::A3UiuJ83
    # model="ft:gpt-4o-mini-2024-07-18:personal:modelo-hexing-fine-tuning:A8rAJQ0R",  # Especifica tu modelo fine-tuned
    messages=messages,
    max_tokens=200
)

response = completion.choices[0].message.content

print(response)

Reporte de Evaluación:
Clave: 1911628
Circuito: RET289
Requiere Orden de Servicio: Sí
Es Recurrente: No
Criticidad: Alta

Justificación: La alerta de "Manually Demand Reset" indica que el medidor ha sido manipulado, lo que puede ser señal de hurto de energía. "Current Unbalance" es un indicador de problemas en la red que podrían afectar la facturación. El hecho de que falte la señal de tensión en L3 sugiere que puede haber un problema de suministro o un medidor dañado, lo que también requiera atención.
